# Kieran Molloy UCDPA Project 
### Course CIDAB 2022-01-18
***

### Environment Information
<table align="left">
<tr>
    <th>Environment Type</th>
    <th>Anaconda Version</th>
    <th>Anaconda Build Channel</th>
    <th>Python Version</th>
</tr>
<tr>
    <td>Anaconda </td>
    <td>2021.11 </td>
    <td>py39_0 </td>
    <td>3.9.7 </td>
</tr>
</table>

## Links to Kaggle Datasource: 
***

#### _[Transactional Retail Dataset of Electronics Store](https://www.kaggle.com/datasets/muhammadshahrayar/transactional-retail-dataset-of-electronics-store)_

*  _[dirty_data.csv](https://www.kaggle.com/datasets/muhammadshahrayar/transactional-retail-dataset-of-electronics-store?select=dirty_data.csv)_
*  _[missing_data.csv](https://www.kaggle.com/datasets/muhammadshahrayar/transactional-retail-dataset-of-electronics-store?select=missing_data.csv)_
*  _[warehouses.csv](https://www.kaggle.com/datasets/muhammadshahrayar/transactional-retail-dataset-of-electronics-store?select=warehouses.csv)_


## Install Modules

In [256]:
# %pip install geopy

## Import Modules
***

In [257]:
from matplotlib import pyplot as plt
from IPython.display import display 

import pandas as pd
import numpy as np
import seaborn as sns

from geopy.distance import geodesic

## Set Variables
***

In [258]:
dirty_data = 'dirty_data.csv'
missing_data = 'missing_data.csv'
warehouses = 'warehouses.csv'

## Functions
***

In [259]:
def get_closest_warehouse(lat , Long ):
    ''' use coordinates to find the closest warehouse '''
    
    # customer coordinates are provided
    coords_customer = lat, Long
    
    # get wharehouse latitude and longitide from warehouse table
    coords_Nickolson = df_warehouses['lat'].loc[df_warehouses.index[0]], df_warehouses['lon'].loc[df_warehouses.index[0]]
    coords_Thompson = df_warehouses['lat'].loc[df_warehouses.index[1]], df_warehouses['lon'].loc[df_warehouses.index[1]]
    coords_Bakers = df_warehouses['lat'].loc[df_warehouses.index[2]], df_warehouses['lon'].loc[df_warehouses.index[2]]
    
    # use geodesic to do a km di
    Nickolson_cust_dist = geodesic(coords_customer, coords_Nickolson).km
    Thompson_cust_dist = geodesic(coords_customer, coords_Thompson).km
    Bakers_cust_dist = geodesic(coords_customer, coords_Bakers).km
    
    val = (Nickolson_cust_dist ,Thompson_cust_dist ,Bakers_cust_dist)
    
    x = val.index(min(val))
    
    if x == 0:
        return('Nickolson')
    elif x == 1:
        return('Thompson')
    else: 
        return('Bakers')


## CSV import 
***

In [260]:
# read the CSV from the local Jupyter Notebook directory 
df_dirty_data = pd.read_csv(dirty_data)
df_missing = pd.read_csv(missing_data)
df_warehouses = pd.read_csv(warehouses)

## Initial CSV Review
***

### df_dirty_data dataframe

In [261]:
type(df_dirty_data)

pandas.core.frame.DataFrame

In [262]:
df_dirty_data.head(10)

,order_id,customer_id,date,nearest_warehouse,shopping_cart,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,season,is_expedited_delivery,distance_to_nearest_warehouse,latest_customer_review,is_happy_customer
0,ORD182494,ID6197211592,2019-06-22,Thompson,"[('Lucent 330S', 1), ('Thunder line', 2), ('iS...",12200,79.89,-37.815105,144.932843,10,11059.89,Winter,True,1.2800,perfect phone and trusted seller. phone itself...,True
1,ORD395518,ID0282825849,2019-12-29,Thompson,"[('Thunder line', 1), ('Universe Note', 2)]",9080,62.71,-37.802736,144.951118,0,9142.71,Summer,False,1.1621,it keeps dropping calls the wifi don't work th...,False
2,ORD494479,ID0579391891,2019-03-02,Nickolson,"[('Thunder line', 1), ('pearTV', 2)]",10670,65.87,-37.821302,144.957581,10,9668.87,Autumn,False,1.0949,five stars this is a great cheap phone.,True
3,ORD019224,ID4544561904,2019-01-12,Nickolson,"[('Universe Note', 1), ('Alcon 10', 2), ('Oliv...",24800,57.61,-37.811416,144.973073,15,21137.61,Summer,False,0.8571,charger did not fit the charger didn't fit.,False
4,ORD104032,ID6231506320,2019-11-28,Nickolson,"[('Universe Note', 1), ('Olivia x460', 1), ('i...",9145,75.54,37.823859,144.969892,25,6934.29,Spring,False,0.5867,four stars good,True
5,ORD146760,ID0311654900,2019-09-16,Bakers,"[('Thunder line', 2), ('Universe Note', 1)]",7810,71.22,37.820245,145.014944,10,7100.22,Spring,False,2.0752,stolen phone sold us a stolen phone so we coul...,False
6,ORD337984,ID3394768956,2019-09-14,Thompson,"[('Candle Inferno', 1), ('Alcon 10', 1), ('Tos...",13700,74.84,-37.807743,144.951570,5,13089.84,Spring,False,0.6767,"love our inferno stick,easy to set up and have...",True
7,ORD072312,ID0774517121,2019-05-23,Thompson,"[('Universe Note', 1), ('Thunder line', 2), ('...",7960,52.28,-37.806337,144.959544,5,10789.79,Autumn,False,1.3043,it sucks mine came with dead pixels,False
8,ORD377837,ID4769265355,2019-10-09,Bakers,"[('Alcon 10', 2), ('Thunder line', 1), ('Candl...",25390,107.58,-37.810810,145.014073,10,22958.58,Spring,True,1.6595,this is how top phone should look like! super ...,True
9,ORD462194,ID5301568579,2019-03-21,Thompson,"[('Universe Note', 1), ('Lucent 330S', 1), ('T...",13320,62.26,-37.808675,144.942337,15,11384.26,winter,True,0.6093,does not live up to its reputation. customer s...,False


In [263]:
# using display for better formatting in Notebook , use print usually
display(df_dirty_data.describe())

,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,distance_to_nearest_warehouse
count,500.000000,500.000000,500.000000,500.000000,500.000000,5.000000e+02,500.000000
mean,25522.216000,76.658200,-35.835234,144.969494,10.890000,3.920967e+04,2.204224
std,86333.729169,14.481465,12.045393,0.022720,8.649134,2.741940e+05,8.812416
min,585.000000,46.350000,-37.827123,144.924967,0.000000,6.392900e+02,0.107800
25%,7050.000000,65.982500,-37.818222,144.953488,5.000000,6.454735e+03,0.751425
50%,12807.500000,76.310000,-37.812165,144.965357,10.000000,1.129396e+04,1.030100
75%,20360.000000,82.555000,-37.805364,144.983985,15.000000,1.811919e+04,1.408625
max,947691.000000,114.040000,37.826339,145.019837,25.000000,5.688270e+06,94.973400


In [264]:
print(df_dirty_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       500 non-null    object 
 1   customer_id                    500 non-null    object 
 2   date                           500 non-null    object 
 3   nearest_warehouse              500 non-null    object 
 4   shopping_cart                  500 non-null    object 
 5   order_price                    500 non-null    int64  
 6   delivery_charges               500 non-null    float64
 7   customer_lat                   500 non-null    float64
 8   customer_long                  500 non-null    float64
 9   coupon_discount                500 non-null    int64  
 10  order_total                    500 non-null    float64
 11  season                         500 non-null    object 
 12  is_expedited_delivery          500 non-null    boo

In [265]:
print(df_dirty_data.isna().any())

order_id                         False
customer_id                      False
date                             False
nearest_warehouse                False
shopping_cart                    False
order_price                      False
delivery_charges                 False
customer_lat                     False
customer_long                    False
coupon_discount                  False
order_total                      False
season                           False
is_expedited_delivery            False
distance_to_nearest_warehouse    False
latest_customer_review           False
is_happy_customer                False
dtype: bool


In [266]:
print(df_dirty_data.isnull().sum())

order_id                         0
customer_id                      0
date                             0
nearest_warehouse                0
shopping_cart                    0
order_price                      0
delivery_charges                 0
customer_lat                     0
customer_long                    0
coupon_discount                  0
order_total                      0
season                           0
is_expedited_delivery            0
distance_to_nearest_warehouse    0
latest_customer_review           0
is_happy_customer                0
dtype: int64


### df_missing_data dataframe

In [267]:
type(df_missing_data)

pandas.core.frame.DataFrame

In [268]:
df_missing_data.head(10)

,order_id,customer_id,date,nearest_warehouse,shopping_cart,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,season,is_expedited_delivery,distance_to_nearest_warehouse,latest_customer_review,is_happy_customer
0,ORD382112,ID0289597187,2019-03-06,Thompson,"[('pearTV', 1), ('Candle Inferno', 2)]",7170.0,61.72,-37.811782,144.951972,15,6156.22,Autumn,True,0.4425,"to many adds in ui it has adds, sends info to ...",False
1,ORD378488,ID1668523020,2019-05-05,Thompson,"[('Thunder line', 1), ('Lucent 330S', 2), ('Al...",13590.0,65.20,-37.808236,144.942181,10,12296.20,Autumn,False,0.6547,s10e haven't had this phone but a short time b...,True
2,ORD279446,ID0370751503,2019-03-24,Bakers,"[('Candle Inferno', 2), ('Universe Note', 1)]",4310.0,78.75,-37.812941,144.985883,15,3742.25,NaN,True,0.8852,best bang for your buck! great overall android...,True
3,ORD277196,ID0634774947,2019-01-12,Thompson,"[('pearTV', 1), ('Thunder line', 1), ('Olivia ...",10940.0,92.59,-37.811748,144.938192,25,8297.59,Summer,True,0.7875,good it was a gift for my niece. it to her in ...,True
4,ORD116193,ID3313210924,2019-09-02,Nickolson,"[('pearTV', 2), ('Universe Note', 2), ('Thunde...",21700.0,95.62,-37.818393,144.967034,5,20710.62,Spring,True,0.2225,"excellent product excellent product, was recei...",True
5,ORD005004,ID0472236192,2019-05-07,Nickolson,"[('Thunder line', 2), ('Lucent 330S', 2)]",6820.0,84.09,-37.801502,144.966104,0,6904.09,Autumn,True,1.9268,i love it very nice and good product would rec...,True
6,ORD296379,ID0591306178,2019-01-19,Nickolson,"[('Thunder line', 1), ('pearTV', 1)]",8490.0,77.51,-37.823700,144.956314,0,8567.51,Summer,False,1.2953,great phone great phone i am still using that ...,True
7,ORD447851,ID0591430562,2019-02-07,NaN,"[('Alcon 10', 2), ('Toshika 750', 2), ('Univer...",32440.0,67.27,-37.817412,144.967590,5,30885.27,Summer,False,0.2169,"five stars it was a gift, my nephew is loving it.",True
8,ORD078449,ID0030287324,2019-10-20,Nickolson,"[('Olivia x460', 2), ('iAssist Line', 2)]",6900.0,105.50,-37.807081,144.967288,15,5970.50,Spring,True,1.2971,definitively would buy again. has been great t...,True
9,ORD018258,ID0245537598,2019-05-27,Nickolson,"[('Universe Note', 1), ('Lucent 330S', 1), ('i...",7205.0,65.39,-37.810164,144.966819,15,6189.64,Autumn,False,0.9688,my best snapchat phone my snapchats are crysta...,True


In [269]:
# using display for better formatting in Notebook , use print usually
display(df_missing_data.describe())

,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,distance_to_nearest_warehouse
count,490.000000,500.000000,490.000000,490.000000,500.000000,490.000000,490.000000
mean,13217.867347,77.683380,-37.812597,144.966114,11.190000,11939.841020,1.077335
std,7552.951453,14.566667,0.007593,0.021054,8.734213,7032.605471,0.509914
min,580.000000,46.200000,-37.828216,144.921217,0.000000,568.640000,0.054900
25%,7140.000000,66.975000,-37.818755,144.951541,5.000000,6486.542500,0.723625
50%,12180.000000,77.275000,-37.813053,144.963409,10.000000,10756.615000,1.043350
75%,18327.500000,85.217500,-37.806714,144.980157,15.000000,16315.555000,1.389525
max,37300.000000,110.990000,-37.788265,145.017015,25.000000,37362.470000,3.138800


In [270]:
print(df_missing_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       500 non-null    object        
 1   customer_id                    500 non-null    object        
 2   date                           500 non-null    datetime64[ns]
 3   nearest_warehouse              490 non-null    object        
 4   shopping_cart                  500 non-null    object        
 5   order_price                    490 non-null    float64       
 6   delivery_charges               500 non-null    float64       
 7   customer_lat                   490 non-null    float64       
 8   customer_long                  490 non-null    float64       
 9   coupon_discount                500 non-null    int64         
 10  order_total                    490 non-null    float64       
 11  season             

In [271]:
print(df_missing_data.isna().any())

order_id                         False
customer_id                      False
date                             False
nearest_warehouse                 True
shopping_cart                    False
order_price                       True
delivery_charges                 False
customer_lat                      True
customer_long                     True
coupon_discount                  False
order_total                       True
season                            True
is_expedited_delivery            False
distance_to_nearest_warehouse     True
latest_customer_review           False
is_happy_customer                 True
dtype: bool


In [272]:
print(df_missing_data.isnull().sum())

order_id                          0
customer_id                       0
date                              0
nearest_warehouse                10
shopping_cart                     0
order_price                      10
delivery_charges                  0
customer_lat                     10
customer_long                    10
coupon_discount                   0
order_total                      10
season                           10
is_expedited_delivery             0
distance_to_nearest_warehouse    10
latest_customer_review            0
is_happy_customer                10
dtype: int64


### df_warehouses dataframe

In [273]:
type(df_warehouses)

pandas.core.frame.DataFrame

In [274]:
df_warehouses.head()

,names,lat,lon
0,Nickolson,-37.818595,144.969551
1,Thompson,-37.812673,144.947069
2,Bakers,-37.809996,144.995232


In [275]:
# using display for better formatting in Notebook , use print usually
display(df_warehouses.describe())

,lat,lon
count,3.000000,3.000000
mean,-37.813755,144.970617
std,0.004400,0.024099
min,-37.818595,144.947069
25%,-37.815634,144.958310
50%,-37.812673,144.969551
75%,-37.811335,144.982392
max,-37.809996,144.995232


In [276]:
print(df_warehouses.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   names   3 non-null      object 
 1   lat     3 non-null      float64
 2   lon     3 non-null      float64
dtypes: float64(2), object(1)
memory usage: 200.0+ bytes
None


In [277]:
print(df_warehouses.isna().any())

names    False
lat      False
lon      False
dtype: bool


In [278]:
print(df_warehouses.isnull().sum())

names    0
lat      0
lon      0
dtype: int64


## Re-import from CSV 
***

In [279]:
# read the CSV from the local Jupyter Notebook directory 
# parsing dates on import since dates were objects
df_dirty_data = pd.read_csv(dirty_data, parse_dates=['date'],dtype={'order_price': 'float'}) 
df_missing_data = pd.read_csv(missing_data, parse_dates=['date']) 
#  cant use dtype={'is_happy_customer': 'bool'} because is_happy_customer contains NaN 

In [280]:
print(df_dirty_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       500 non-null    object        
 1   customer_id                    500 non-null    object        
 2   date                           500 non-null    datetime64[ns]
 3   nearest_warehouse              500 non-null    object        
 4   shopping_cart                  500 non-null    object        
 5   order_price                    500 non-null    float64       
 6   delivery_charges               500 non-null    float64       
 7   customer_lat                   500 non-null    float64       
 8   customer_long                  500 non-null    float64       
 9   coupon_discount                500 non-null    int64         
 10  order_total                    500 non-null    float64       
 11  season             

In [281]:
print(df_missing_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       500 non-null    object        
 1   customer_id                    500 non-null    object        
 2   date                           500 non-null    datetime64[ns]
 3   nearest_warehouse              490 non-null    object        
 4   shopping_cart                  500 non-null    object        
 5   order_price                    490 non-null    float64       
 6   delivery_charges               500 non-null    float64       
 7   customer_lat                   490 non-null    float64       
 8   customer_long                  490 non-null    float64       
 9   coupon_discount                500 non-null    int64         
 10  order_total                    490 non-null    float64       
 11  season             

## Detect and fix errors in dirty_data
***

## Fix missing values in missing_data
***

In [283]:
# checking the missing values for nearest_warehouse
df_missing_data[df_missing_data['nearest_warehouse'].isnull()]

,order_id,customer_id,date,nearest_warehouse,shopping_cart,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,season,is_expedited_delivery,distance_to_nearest_warehouse,latest_customer_review,is_happy_customer
7,ORD447851,ID0591430562,2019-02-07,NaN,"[('Alcon 10', 2), ('Toshika 750', 2), ('Univer...",32440.0,67.27,-37.817412,144.967590,5,30885.27,Summer,False,0.2169,"five stars it was a gift, my nephew is loving it.",True
16,ORD256447,ID1404216319,2019-12-13,NaN,"[('Candle Inferno', 1), ('Olivia x460', 1), ('...",6015.0,77.47,-37.801138,144.988605,25,4588.72,Summer,False,1.1454,i love this phone i love this phone. very fast...,True
105,ORD159597,ID2632208068,2019-08-18,NaN,"[('Lucent 330S', 2), ('Candle Inferno', 2), ('...",9630.0,80.40,-37.819355,144.989577,10,8747.40,Winter,True,1.1544,quality love it,True
219,ORD474381,ID2776391626,2019-12-11,NaN,"[('Olivia x460', 2), ('Thunder line', 2), ('Ca...",7670.0,75.06,-37.810971,144.963102,25,5827.56,Summer,False,1.0207,what's not to like? this phone does everything...,True
236,ORD372144,ID0814520160,2019-12-11,NaN,"[('iAssist Line', 2), ('Olivia x460', 1), ('Al...",18945.0,96.53,-37.812604,144.957172,15,16199.78,Summer,True,0.8885,"it’s good, but it’s not compatible with my car...",True
247,ORD052805,ID4234925766,2019-05-19,NaN,"[('Universe Note', 2), ('Candle Inferno', 2), ...",9940.0,68.65,-37.804575,144.967468,15,8517.65,Autumn,True,1.5714,accessories not working properly. i have used ...,False
251,ORD222346,ID0368376953,2019-10-22,NaN,"[('Olivia x460', 2), ('Universe Note', 1), ('L...",8360.0,81.02,-37.802945,144.981699,0,8441.02,Spring,False,1.4257,i liked thank you but don't have headphones 🤔 ...,True
292,ORD229269,ID0582156569,2019-12-31,NaN,"[('Alcon 10', 1), ('pearTV', 1), ('iAssist Lin...",21805.0,97.96,-37.788265,144.929202,25,16451.71,Summer,False,3.1388,great product. great product came as described...,True
295,ORD433678,ID0176736472,2019-09-04,NaN,"[('Olivia x460', 1), ('iAssist Line', 1), ('pe...",16070.0,97.81,-37.814120,144.968115,0,16167.81,Spring,True,0.5139,amazing!,True
416,ORD075737,ID2702095693,2019-10-22,NaN,"[('iAssist Line', 2), ('pearTV', 2), ('Toshika...",25710.0,73.29,-37.810628,144.990476,15,21926.79,Spring,False,0.4241,then this phone is great for drawing on the go...,True


In [ ]:
get_closest_warehouse(-37.821302,144.957581)

## Analyse
***

## Visualise
***